In [2]:
print("hi")

hi


# To Start!

We first import the packages that we will be using. 

We need pandas to work on our data, matplotlib pyplot for visualizations, numpy for array calculations, and finally networkx for all our graph networks.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from numba import njit
import random
import pandas as pd
import BowenProjectFunctions as bpf

Next, we grab the static gtfs rail text files and read them into dataframes.

In [ ]:
trips = pd.read_csv('../gtfs_rail/trips.txt')
stops = pd.read_csv('../gtfs_rail/stops.txt')